### 构建路由链

In [1]:

from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("api_key")
base_url = os.getenv("base_url")

online_llm = ChatOpenAI(
    api_key=api_key,
    model="qwen-plus",
    base_url=base_url,
    temperature=0
)




In [2]:
# 路由链相关环境
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.router import MultiRouteChain, LLMRouterChain, MultiPromptChain
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

In [3]:
# 底层 目标链的prompt
math_prompt = """
你是一位数学家，请根据以下问题，生成对应的数学公式.内容：{input}
"""

weather_prompt = """你是一位天气预报员，请根据以下问题，生成对应的天气信息.内容：{input}"""
news_prompt = """你是一位新闻工作者，请根据以下问题，生成对应的新闻内容.内容：{input}"""

In [4]:
#提示词模版 相关信息
prompt_info = [
    {
        "name": "math",
        "description": "数学问题",
        "prompt": math_prompt
    },
    {
        "name": "weather",
        "description": "天气问题",
        "prompt": weather_prompt
    },
    {
        "name": "news",
        "description": "新闻问题",
        "prompt": news_prompt
    }
]

In [5]:
#定义 目标链
chains = {}
for info in prompt_info:
    chain = LLMChain(llm=online_llm, prompt=PromptTemplate.from_template(template=info["prompt"]))
    chains[info["name"]] = chain

print(chains)

{'math': LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='\n你是一位数学家，请根据以下问题，生成对应的数学公式.内容：{input}\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10c73f050>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10c8b4d50>, root_client=<openai.OpenAI object at 0x10bf8ead0>, root_async_client=<openai.AsyncOpenAI object at 0x10bcc58d0>, model_name='qwen-plus', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://dashscope.aliyuncs.com/compatible-mode/v1'), output_parser=StrOutputParser(), llm_kwargs={}), 'weather': LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='你是一位天气预报员，请根据以下问题，生成对应的天气信息.内容：{input}'), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10c73f050>, async_clien

/var/folders/xh/fl5j9g1n5t79htj8yk1cswzh0000gn/T/ipykernel_21574/2015125349.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=online_llm, prompt=PromptTemplate.from_template(template=info["prompt"]))


In [22]:
#定义默认链提示词
default_prompt = """你是一位助手，别人无法回答的问题，你来回答，请回答以下问题：{input}"""

In [39]:
#定义默认链
default_chain = LLMChain(llm=online_llm, prompt=PromptTemplate.from_template(template=default_prompt))

In [24]:
#路由配置
prompt_info = [
    "math:你是一位数学家，请根据以下问题，生成对应的数学公式",
    "weather:你是一位天气预报员，请根据以下问题，生成对应的天气信息",
    "news:你是一位新闻工作者，请根据以下问题，生成对应的新闻内容",
]

In [26]:
#构建路由模版
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations = "\n".join(prompt_info))
print("路由模版是：",router_template)

router_prompt = PromptTemplate(
    template=router_template,
    # template="\n".join(prompt_info),
    input_variables=["input"],
    output_parser=RouterOutputParser()
    )
print("路由模版router_prompt 是：",router_prompt)

路由模版是： Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
math:你是一位数学家，请根据以下问题，生成对应的数学公式
weather:你是一位天气预报员，请根据以下问题，生成对应的天气信息
news:你是一位新闻工作者，请根据以下问题，生成对应

In [ ]:
#构建路由链
router_chain = LLMRouterChain.from_llm(llm=online_llm, prompt=router_prompt, verbose=True)

In [40]:
#初始化记忆模块
memory = ConversationBufferMemory(input_key="input", output_key="text", return_messages=True)

In [41]:
#构建最终路由链，将路由链和默认链组合, 并添加记忆模块
expert_chain = MultiPromptChain(
    router_chain=router_chain, #路由链
    destination_chains=chains, #路由链对应的链
    default_chain=default_chain, #默认链
    verbose=True,
    memory=memory, #记忆模块
)


In [42]:
#调用最终链

question = [
    "你叫什么名字？",
    "请你写一首诗，歌颂祖国"
]

for q in question:
    print("问题：",q)
    print("回答：",expert_chain.invoke({"input":q}))

问题： 你叫什么名字？


> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
None: {'input': '你叫什么名字？'}
> Finished chain.
回答： {'input': '你叫什么名字？', 'history': [HumanMessage(content='你叫什么名字？', additional_kwargs={}, response_metadata={}), AIMessage(content='我叫通义千问，英文名是Qwen。我是阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。很高兴为您服务！如果您有任何问题，欢迎随时向我提问。', additional_kwargs={}, response_metadata={})], 'text': '我叫通义千问，英文名是Qwen。我是阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。很高兴为您服务！如果您有任何问题，欢迎随时向我提问。'}
问题： 请你写一首诗，歌颂祖国


> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
None: {'input': '请你写一首诗，歌颂祖国'}
> Finished chain.
回答： {'input': '请你写一首诗，歌颂祖国', 'history': [HumanMessage(content='你叫什么名字？', additional_kwargs={}, response_metadata={}), AIMessage(content='我叫通义千问，英文名是Qwen。我是阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。很高兴为您服务！如果您有任何问题，欢迎随时向我提问。', additional_kwargs={}, response_metadata={}), HumanMessage(content='请你写一首诗，歌颂祖国', additional_kwargs={}, response_metadata={

### 查看 路由链对话记忆

In [44]:
expert_chain.memory.chat_memory.messages

[HumanMessage(content='你叫什么名字？', additional_kwargs={}, response_metadata={}),
 AIMessage(content='我叫通义千问，英文名是Qwen。我是阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。很高兴为您服务！如果您有任何问题，欢迎随时向我提问。', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='请你写一首诗，歌颂祖国', additional_kwargs={}, response_metadata={}),
 AIMessage(content='以下是几首不同风格的歌颂祖国的诗，供你参考：\n\n### 古风豪迈型\n**《赞华夏》**\n\n巍巍华夏五千年，壮丽山河谱锦篇。\n秦月汉关凝浩气，唐风宋韵铸华笺。\n长城万里雄心在，大漠千秋浩梦传。\n今日龙腾惊四海，神州处处绽新颜。\n\n解析：首联点明中华民族悠久的历史和灿烂辉煌的文化篇章。颔联通过“秦月汉关”“唐风宋韵”具体展现古代中国的辉煌成就与文化积淀。颈联以长城、大漠象征着中华民族坚韧不拔的精神和绵延不绝的梦想传承。尾联则描绘出当今中国蓬勃发展的态势，令世界瞩目，到处呈现出崭新的面貌。\n\n### 清新典雅型\n**《国之韵》**\n\n春临禹甸柳丝柔，碧水青山映眼眸。\n古寺钟声传雅韵，新城灯火耀金瓯。\n田园牧曲添佳趣，科技繁花竞自由。\n盛世祥和民乐业，家国同梦共春秋。\n\n解析：开篇描绘春天来临，大地一片生机勃勃，柔和的柳丝与清澈的山水相映成趣。接着写到古老的寺庙钟声传递着高雅的韵味，而现代化的城市灯火照亮了整个国家（金瓯常用来比喻国土）。然后提到乡村田园生活充满乐趣，同时科技发展如繁花般竞相绽放，给人带来无限可能。最后强调在盛世中人民安居乐业，共同怀揣着家国梦想，一起走过岁月长河。\n\n### 激昂奋进型\n**《颂中华崛起》**\n\n烽火曾经岁月稠，英雄热血写春秋。\n改革潮涌开新纪，开放帆扬逐远流。\n科技兴邦强国路，民生为本固金瓯。\n复兴路上雄心在，再创辉煌万代留。\n\n解析：首联回顾过去，指出曾经经历过无数战火纷飞的岁月，无数英雄用热血书写了壮丽的历史篇章。颔联阐述改革开放这一伟大决策开启了

In [47]:
print(expert_chain.memory.chat_memory.messages[-1].content)

以下是几首不同风格的歌颂祖国的诗，供你参考：

### 古风豪迈型
**《赞华夏》**

巍巍华夏五千年，壮丽山河谱锦篇。
秦月汉关凝浩气，唐风宋韵铸华笺。
长城万里雄心在，大漠千秋浩梦传。
今日龙腾惊四海，神州处处绽新颜。

解析：首联点明中华民族悠久的历史和灿烂辉煌的文化篇章。颔联通过“秦月汉关”“唐风宋韵”具体展现古代中国的辉煌成就与文化积淀。颈联以长城、大漠象征着中华民族坚韧不拔的精神和绵延不绝的梦想传承。尾联则描绘出当今中国蓬勃发展的态势，令世界瞩目，到处呈现出崭新的面貌。

### 清新典雅型
**《国之韵》**

春临禹甸柳丝柔，碧水青山映眼眸。
古寺钟声传雅韵，新城灯火耀金瓯。
田园牧曲添佳趣，科技繁花竞自由。
盛世祥和民乐业，家国同梦共春秋。

解析：开篇描绘春天来临，大地一片生机勃勃，柔和的柳丝与清澈的山水相映成趣。接着写到古老的寺庙钟声传递着高雅的韵味，而现代化的城市灯火照亮了整个国家（金瓯常用来比喻国土）。然后提到乡村田园生活充满乐趣，同时科技发展如繁花般竞相绽放，给人带来无限可能。最后强调在盛世中人民安居乐业，共同怀揣着家国梦想，一起走过岁月长河。

### 激昂奋进型
**《颂中华崛起》**

烽火曾经岁月稠，英雄热血写春秋。
改革潮涌开新纪，开放帆扬逐远流。
科技兴邦强国路，民生为本固金瓯。
复兴路上雄心在，再创辉煌万代留。

解析：首联回顾过去，指出曾经经历过无数战火纷飞的岁月，无数英雄用热血书写了壮丽的历史篇章。颔联阐述改革开放这一伟大决策开启了新的时代篇章，推动着国家向前发展。颈联说明依靠科技进步来实现国家强盛的道路，并且始终把民生放在首位以稳固国家根基。尾联表达了在复兴道路上我们有着坚定的雄心壮志，要再次创造辉煌业绩流传万代。
